In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

In [2]:
os.environ['SPARK_HOME'] = r'C:\spark\spark-3.5.4-bin-hadoop3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
spark = (
    SparkSession
    .builder
    .appName("YouTube")
    .master("local[*]")
    .getOrCreate()
)

In [4]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

In [6]:
emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [7]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [15]:
emp_gender_fixed = emp.withColumn(
    'new_gender',
    F.when(F.col('gender') == 'Male', 'M')
    .when(F.col('gender') == 'Female', 'F')
    .otherwise(None)
)

In [10]:
emp_gendered_fixed.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+----------+---+------+------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M|
+-----------+-------------+----------+---+------+------+----------+----------+
only showing top 5 rows



In [13]:
emp_gender_fixed_1 = emp.withColumn(
    'new_gender', F.expr("CASE WHEN gender = 'Male' THEN 'M' WHEN gender = 'Female' THEN 'F' ELSE NULL END")
)

In [14]:
emp_gender_fixed_1.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+----------+---+------+------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M|
+-----------+-------------+----------+---+------+------+----------+----------+
only showing top 5 rows



In [16]:
emp_name_fixed = emp_gender_fixed.withColumn(
    'new_name', F.regexp_replace('name', 'J', 'Z')
)

In [17]:
emp_name_fixed.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|  new_name|
+-----------+-------------+----------+---+------+------+----------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|  Zohn Doe|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|Zane Smith|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M| Bob Brown|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F| Alice Lee|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M| Zack Chan|
+-----------+-------------+----------+---+------+------+----------+----------+----------+
only showing top 5 rows



In [18]:
emp_data_fix = emp_name_fixed.withColumn(
    'hire_date', F.to_date(F.col('hire_date'), format='yyyy-MM-dd')
)

In [19]:
emp_data_fix.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|  new_name|
+-----------+-------------+----------+---+------+------+----------+----------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|  Zohn Doe|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|Zane Smith|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M| Bob Brown|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F| Alice Lee|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M| Zack Chan|
+-----------+-------------+----------+---+------+------+----------+----------+----------+
only showing top 5 rows



In [20]:
emp_data_fix.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- new_gender: string (nullable = true)
 |-- new_name: string (nullable = true)



In [21]:
emp_dated = emp_data_fix.withColumn(
    'date_now', F.current_date()
).withColumn(
    'time_stamp_now', F.current_timestamp()
)

In [25]:
emp_dated.show(5, truncate=False)

+-----------+-------------+----------+---+------+------+----------+----------+----------+----------+--------------------------+
|employee_id|department_id|name      |age|gender|salary|hire_date |new_gender|new_name  |date_now  |time_stamp_now            |
+-----------+-------------+----------+---+------+------+----------+----------+----------+----------+--------------------------+
|001        |101          |John Doe  |30 |Male  |50000 |2015-01-01|M         |Zohn Doe  |2025-01-18|2025-01-18 10:38:33.570391|
|002        |101          |Jane Smith|25 |Female|45000 |2016-02-15|F         |Zane Smith|2025-01-18|2025-01-18 10:38:33.570391|
|003        |102          |Bob Brown |35 |Male  |55000 |2014-05-01|M         |Bob Brown |2025-01-18|2025-01-18 10:38:33.570391|
|004        |102          |Alice Lee |28 |Female|48000 |2017-09-30|F         |Alice Lee |2025-01-18|2025-01-18 10:38:33.570391|
|005        |103          |Jack Chan |40 |Male  |60000 |2013-04-01|M         |Zack Chan |2025-01-18|2025

In [26]:
emp_1 = emp_dated.na.drop()

In [27]:
emp_1.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+----------+----------+--------------------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|  new_name|  date_now|      time_stamp_now|
+-----------+-------------+----------+---+------+------+----------+----------+----------+----------+--------------------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|  Zohn Doe|2025-01-18|2025-01-18 10:39:...|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|Zane Smith|2025-01-18|2025-01-18 10:39:...|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M| Bob Brown|2025-01-18|2025-01-18 10:39:...|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F| Alice Lee|2025-01-18|2025-01-18 10:39:...|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M| Zack Chan|2025-01-18|2025-01-18 10:39:...|
+-----------+-----------

In [28]:
emp_null_df = emp_dated.withColumn(
    'new_gender', F.coalesce(F.col('new_gender'), F.lit('O'))
)

In [29]:
emp_null_df.show(5)

+-----------+-------------+----------+---+------+------+----------+----------+----------+----------+--------------------+
|employee_id|department_id|      name|age|gender|salary| hire_date|new_gender|  new_name|  date_now|      time_stamp_now|
+-----------+-------------+----------+---+------+------+----------+----------+----------+----------+--------------------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|         M|  Zohn Doe|2025-01-18|2025-01-18 10:41:...|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|         F|Zane Smith|2025-01-18|2025-01-18 10:41:...|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|         M| Bob Brown|2025-01-18|2025-01-18 10:41:...|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|         F| Alice Lee|2025-01-18|2025-01-18 10:41:...|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|         M| Zack Chan|2025-01-18|2025-01-18 10:41:...|
+-----------+-----------

In [30]:
emp_final = emp_null_df.drop(
    'name', 'gender'
).withColumnRenamed(
    'new_name', 'name'
).withColumnRenamed(
    'new_gender', 'gender'
)

In [31]:
emp_final.show(5)

+-----------+-------------+---+------+----------+------+----------+----------+--------------------+
|employee_id|department_id|age|salary| hire_date|gender|      name|  date_now|      time_stamp_now|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+
|        001|          101| 30| 50000|2015-01-01|     M|  Zohn Doe|2025-01-18|2025-01-18 10:46:...|
|        002|          101| 25| 45000|2016-02-15|     F|Zane Smith|2025-01-18|2025-01-18 10:46:...|
|        003|          102| 35| 55000|2014-05-01|     M| Bob Brown|2025-01-18|2025-01-18 10:46:...|
|        004|          102| 28| 48000|2017-09-30|     F| Alice Lee|2025-01-18|2025-01-18 10:46:...|
|        005|          103| 40| 60000|2013-04-01|     M| Zack Chan|2025-01-18|2025-01-18 10:46:...|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+
only showing top 5 rows



In [32]:
emp_fixed = emp_final.withColumn(
    'date_string', F.date_format(F.col('hire_date'), 'dd/MM/yyyy')
)

In [33]:
emp_fixed.show(5)

+-----------+-------------+---+------+----------+------+----------+----------+--------------------+-----------+
|employee_id|department_id|age|salary| hire_date|gender|      name|  date_now|      time_stamp_now|date_string|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+-----------+
|        001|          101| 30| 50000|2015-01-01|     M|  Zohn Doe|2025-01-18|2025-01-18 10:48:...| 01/01/2015|
|        002|          101| 25| 45000|2016-02-15|     F|Zane Smith|2025-01-18|2025-01-18 10:48:...| 15/02/2016|
|        003|          102| 35| 55000|2014-05-01|     M| Bob Brown|2025-01-18|2025-01-18 10:48:...| 01/05/2014|
|        004|          102| 28| 48000|2017-09-30|     F| Alice Lee|2025-01-18|2025-01-18 10:48:...| 30/09/2017|
|        005|          103| 40| 60000|2013-04-01|     M| Zack Chan|2025-01-18|2025-01-18 10:48:...| 01/04/2013|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+-----

In [35]:
emp_year = emp_final.withColumn(
    'year', F.date_format(F.col('hire_date'), 'yyyy')
)

In [36]:
emp_year.show(5)

+-----------+-------------+---+------+----------+------+----------+----------+--------------------+----+
|employee_id|department_id|age|salary| hire_date|gender|      name|  date_now|      time_stamp_now|year|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+----+
|        001|          101| 30| 50000|2015-01-01|     M|  Zohn Doe|2025-01-18|2025-01-18 10:50:...|2015|
|        002|          101| 25| 45000|2016-02-15|     F|Zane Smith|2025-01-18|2025-01-18 10:50:...|2016|
|        003|          102| 35| 55000|2014-05-01|     M| Bob Brown|2025-01-18|2025-01-18 10:50:...|2014|
|        004|          102| 28| 48000|2017-09-30|     F| Alice Lee|2025-01-18|2025-01-18 10:50:...|2017|
|        005|          103| 40| 60000|2013-04-01|     M| Zack Chan|2025-01-18|2025-01-18 10:50:...|2013|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+----+
only showing top 5 rows



In [37]:
emp_month = emp_final.withColumn(
    'month', F.date_format(F.col('hire_date'), 'MM')
)

In [38]:
emp_month.show(5)

+-----------+-------------+---+------+----------+------+----------+----------+--------------------+-----+
|employee_id|department_id|age|salary| hire_date|gender|      name|  date_now|      time_stamp_now|month|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+-----+
|        001|          101| 30| 50000|2015-01-01|     M|  Zohn Doe|2025-01-18|2025-01-18 10:50:...|   01|
|        002|          101| 25| 45000|2016-02-15|     F|Zane Smith|2025-01-18|2025-01-18 10:50:...|   02|
|        003|          102| 35| 55000|2014-05-01|     M| Bob Brown|2025-01-18|2025-01-18 10:50:...|   05|
|        004|          102| 28| 48000|2017-09-30|     F| Alice Lee|2025-01-18|2025-01-18 10:50:...|   09|
|        005|          103| 40| 60000|2013-04-01|     M| Zack Chan|2025-01-18|2025-01-18 10:50:...|   04|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+-----+
only showing top 5 rows



In [39]:
emp_day = emp_final.withColumn(
    'day', F.date_format(F.col('hire_date'), 'dd')
)

In [40]:
emp_day.show(5)

+-----------+-------------+---+------+----------+------+----------+----------+--------------------+---+
|employee_id|department_id|age|salary| hire_date|gender|      name|  date_now|      time_stamp_now|day|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+---+
|        001|          101| 30| 50000|2015-01-01|     M|  Zohn Doe|2025-01-18|2025-01-18 10:51:...| 01|
|        002|          101| 25| 45000|2016-02-15|     F|Zane Smith|2025-01-18|2025-01-18 10:51:...| 15|
|        003|          102| 35| 55000|2014-05-01|     M| Bob Brown|2025-01-18|2025-01-18 10:51:...| 01|
|        004|          102| 28| 48000|2017-09-30|     F| Alice Lee|2025-01-18|2025-01-18 10:51:...| 30|
|        005|          103| 40| 60000|2013-04-01|     M| Zack Chan|2025-01-18|2025-01-18 10:51:...| 01|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+---+
only showing top 5 rows



In [43]:
emp_zone = emp_final.withColumn(
    'zone', F.date_format(F.col('time_stamp_now'), 'z')
)

In [45]:
emp_zone.show(5)

+-----------+-------------+---+------+----------+------+----------+----------+--------------------+----+
|employee_id|department_id|age|salary| hire_date|gender|      name|  date_now|      time_stamp_now|zone|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+----+
|        001|          101| 30| 50000|2015-01-01|     M|  Zohn Doe|2025-01-18|2025-01-18 10:53:...| EET|
|        002|          101| 25| 45000|2016-02-15|     F|Zane Smith|2025-01-18|2025-01-18 10:53:...| EET|
|        003|          102| 35| 55000|2014-05-01|     M| Bob Brown|2025-01-18|2025-01-18 10:53:...| EET|
|        004|          102| 28| 48000|2017-09-30|     F| Alice Lee|2025-01-18|2025-01-18 10:53:...| EET|
|        005|          103| 40| 60000|2013-04-01|     M| Zack Chan|2025-01-18|2025-01-18 10:53:...| EET|
+-----------+-------------+---+------+----------+------+----------+----------+--------------------+----+
only showing top 5 rows

